In [106]:
import psycopg2
from psycopg2 import sql
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import cvxpy as cp

from util import *

In [107]:
problems_by_report = get_problem_data_by_report()

# Выводим результат
for report_id, problems in problems_by_report.items():
    print(f"\nОтчет #{report_id}:")
    for i, (name, freq) in enumerate(problems, 1):
        print(f"  {i}. {name} (частота: {freq})")

# Или в виде списка кортежей
problems_list = get_problem_data_as_list()
print(f'problems_list = {problems_list[0][1]}')
print("\nДанные в виде списка кортежей:")
for report_id, problems in problems_list:
    print(f"Отчет {report_id}: {len(problems)} проблем")


Отчет #1:
  1. Безработица (частота: 80)
  2. Дорожные пробки (частота: 200)
  3. Низкое качество образования (частота: 150)
  4. Коррупция в ЖКХ (частота: 96)
  5. Отсутствие детских садов (частота: 100)
  6. Загрязнение парков (частота: 330)

Отчет #2:
  1. Высокие тарифы ЖКХ (частота: 300)
  2. Недостаток больниц (частота: 491)
  3. Уличное воровство (частота: 73)
  4. Наркомания среди молодежи (частота: 90)
  5. Разрушение дорожного покрытия (частота: 427)
  6. Отсутствие спортивных площадок (частота: 212)
  7. Шумовое загрязнение ночью (частота: 346)
  8. Нехватка парковочных мест (частота: 452)
  9. Проблемы с общественным транспортом (частота: 500)
  10. Засилье рекламных конструкций (частота: 100)
problems_list = [('Безработица', 80), ('Дорожные пробки', 200), ('Низкое качество образования', 150), ('Коррупция в ЖКХ', 96), ('Отсутствие детских садов', 100), ('Загрязнение парков', 330)]

Данные в виде списка кортежей:
Отчет 1: 6 проблем
Отчет 2: 10 проблем


In [108]:
budget_items = get_budget_items()

# Выводим результат
print(f"Получено {len(budget_items)} статей бюджета:")
for i, (name, min_budget) in enumerate(budget_items, 1):
    print(f"{i}. {name} (мин. бюджет: {min_budget:,.2f} руб.)")

# Пример использования вектора
print("\nВектор кортежей:")
print(budget_items)

Получено 15 статей бюджета:
1. топливно-энергетический комплекс (мин. бюджет: 0.00 руб.)
2. водное хозяйство (мин. бюджет: 0.00 руб.)
3. лесное хозяйство (мин. бюджет: 0.00 руб.)
4. транспорт (мин. бюджет: 0.00 руб.)
5. дорожное хозяйство (мин. бюджет: 0.00 руб.)
6. связь и информатика (мин. бюджет: 0.00 руб.)
7. жилищное хозяйство (мин. бюджет: 0.00 руб.)
8. коммунальное хозяйство (мин. бюджет: 0.00 руб.)
9. благоустройство (мин. бюджет: 0.00 руб.)
10. дошкольное образование (мин. бюджет: 0.00 руб.)
11. общее образование (мин. бюджет: 0.00 руб.)
12. среднее профессиональное образование (мин. бюджет: 0.00 руб.)
13. высшее образование (мин. бюджет: 0.00 руб.)
14. стационарная медицинская помощь (мин. бюджет: 0.00 руб.)
15. скорая медицинская помощь (мин. бюджет: 0.00 руб.)

Вектор кортежей:
[('топливно-энергетический комплекс', 0.0), ('водное хозяйство', 0.0), ('лесное хозяйство', 0.0), ('транспорт', 0.0), ('дорожное хозяйство', 0.0), ('связь и информатика', 0.0), ('жилищное хозяйство',

In [109]:
all_matrices = get_influence_matrix_by_report()

# Выводим информацию по каждому отчету
for report_id, (matrix, budget_items, problem_items) in all_matrices.items():
    print(f"\nОтчет #{report_id}")
    print(f"Статьи бюджета: {budget_items}")
    print(f"Проблемы: {problem_items}")
    print("Матрица влияния:")
    print(matrix)
    matrix.to_csv(f"influence_matrix_report_{report_id}.csv")


Отчет #1
Статьи бюджета: ['благоустройство', 'водное хозяйство', 'высшее образование', 'дорожное хозяйство', 'дошкольное образование', 'жилищное хозяйство', 'коммунальное хозяйство', 'лесное хозяйство', 'общее образование', 'связь и информатика', 'скорая медицинская помощь', 'среднее профессиональное образование', 'стационарная медицинская помощь', 'топливно-энергетический комплекс', 'транспорт']
Проблемы: ['Безработица', 'Дорожные пробки', 'Загрязнение парков', 'Коррупция в ЖКХ', 'Низкое качество образования', 'Отсутствие детских садов']
Матрица влияния:
problem_item                          Безработица  Дорожные пробки  \
budget_item                                                          
благоустройство                               0.0              0.0   
водное хозяйство                              0.0              0.0   
высшее образование                            1.0              0.0   
дорожное хозяйство                            0.0              0.0   
дошкольное образо

In [110]:
import matplotlib as plt

def perform_foundation_distribution(B, problem_report_id, distro_func=dp_budget_allocation, *distro_args):
    matrix, budget_labels, problem_labels = get_influence_matrix_for_report(problem_report_id)

    problems_by_rep = get_problem_data_by_report()[problem_report_id]
    budget_items = get_budget_items()

    A = matrix.to_numpy().T
    L = np.array([B / len(A[0]) / 5] * len(A[0]))
    c = np.array([p[1] for p in problems_by_rep])

    result, mae, mse, rmse = distro_func(c, B, L, A, *distro_args)

    print(f'Распределение методом дин программирования = {result}')
    print(f'B sum = {np.sum(result)}')
    print(f"Средняя абсолютная ошибка (mae): {mae / B:.2f}")
    print(f"Среднеквадратичная ошибка (mse): {mse:.4f}")
    print(f"Среднеквадратичная ошибка (rmse): {rmse:.2f}")



In [113]:
def distribute_budget(c, B, L, A, mu=0.0):
    """
    Распределяет бюджетные средства на основе частоты упоминаний проблем и нижних пороговых ограничений.

    Параметры:
    n (int): число проблем
    m (int): число статей расходов
    w (np.array): веса статей
    c (np.array): частоты упоминаний проблем
    B (float): общий бюджет
    L (np.array): нижние пороговые ограничения для статей расходов
    a (np.array): матрица коэффициентов связи между проблемами и статьями расходов

    Возвращает:
    np.array: оптимальное распределение бюджета по статьям расходов
    """
    n = len(c)
    m = len(L)

    # Переменные для оптимизации
    x = cp.Variable(m, nonneg=True)
    l = L / sum(L)

    # y = c / c.sum()
    y = c / c.sum() * B

    # Целевая функция
    I = A @ x - y
    # I = A @ x * B - y

    loss = cp.sum_squares(I)
    reg_loss = cp.sum_squares(x)

    # objective = cp.Minimize(loss)
    objective = cp.Minimize(loss + mu * reg_loss)

    # Ограничения
    constraints = [
        cp.sum(x) == B,
        x >= L,
    ]
    # Задача оптимизации
    problem = cp.Problem(objective, constraints)

    # Решение задачи
    problem.solve(solver=cp.OSQP)
    # problem.solve(verbose=True)

    # Проверка статуса решения
    if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        x_v = x.value
        e = A @ x_v - y

        mae = np.mean(np.abs(e))
        mse = np.mean(e ** 2)
        rmse = np.sqrt(np.mean(e ** 2))

        return x_v, mae, mse, rmse
    else:
        return None


def dp_budget_allocation(c, B, L, A, K=100):
    """
    Приближение L2‑задачи через DP с дискретизацией:
      x_j = k_j * δ,  δ = B/K,  sum k_j = K.

    Возвращает вектор x длины m.
    """
    n, m = A.shape
    # 1) Целевые абсолютные расходы по проблемам
    y = (c / c.sum()) * B
    # 2) Распределяем эти расходы по статьям
    y_art = A.T @ y   # в идеале x_j ~ y_art[j]

    d = B / K
    # минимум гранул на j-ю статью
    kmin = np.ceil(L/d).astype(int)

    # DP таблица
    dp = np.full((m+1, K+1), np.inf)
    dp[0,0] = 0
    # back‑pointer
    parent = np.zeros((m+1, K+1), int)

    for j in range(1, m+1):
        for k in range(K+1):
            for q in range(kmin[j-1], k+1):
                prev = dp[j-1, k-q]
                cost = (q*d - y_art[j-1])**2
                val = prev + cost
                if val < dp[j, k]:
                    dp[j, k] = val
                    parent[j, k] = q

    # восстанавливаем k_j
    k = K
    ks = np.zeros(m, int)
    for j in range(m, 0, -1):
        q = parent[j, k]
        ks[j-1] = q
        k -= q

    # финальное распределение
    x = ks * d

    e = A @ x - y

    mae = np.mean(np.abs(e))
    mse = np.mean(e ** 2)
    rmse = np.sqrt(np.mean(e ** 2))

    return x, mae, mse, rmse

In [114]:
for problem_report_id, problems in problems_by_report.items():

    matrix, budget_labels, problem_labels = get_influence_matrix_for_report(problem_report_id)

    problems_by_rep = get_problem_data_by_report()[problem_report_id]
    budget_items = get_budget_items()

    A = matrix.to_numpy().T
    L = np.array([B / len(A[0]) / 5] * len(A[0]))
    c = np.array([p[1] for p in problems_by_rep])

    result, mae, mse, rmse = distro_func(c, B, L, A, *distro_args)

    print(f'Распределение методом дин программирования = {result}')
    print(f'B sum = {np.sum(result)}')
    print(f"Средняя абсолютная ошибка (mae): {mae / B:.2f}")
    print(f"Среднеквадратичная ошибка (mse): {mse:.4f}")
    print(f"Среднеквадратичная ошибка (rmse): {rmse:.2f}")

распределение методом дин программирования = [136903.76569038  13333.33333333  63682.0083682   13333.33333333
 325188.28451883  80418.41004184  13333.33333333  13333.33333333
  84602.51046025  13333.33333333  13333.33333333  13333.33333333
  13333.33333333  13333.33333333 189205.0209205 ]
B sum = 1000000.0
Средняя абсолютная ошибка (mae): 0.02
Среднеквадратичная ошибка (mse): 400000000.0000
Среднеквадратичная ошибка (rmse): 20000.00
распределение методом дин программирования = [117519.22433969  48311.60147108  48311.60147108 146940.8224674
  48311.60147108 100300.90270812  48311.60147108  48311.60147108
  95787.36208626  48311.60147108  48311.60147108  48311.60147108
  30090.27081244  48311.60147108  74557.00434637]
B sum = 1000000.0
Средняя абсолютная ошибка (mae): 0.03
Среднеквадратичная ошибка (mse): 1795819867.9399
Среднеквадратичная ошибка (rmse): 42377.11


In [112]:
# Получаем данные

# Получаем все матрицы по отчетам


# Получаем матрицу для конкретного отчета
report_id = 2
matrix, budget_items, problem_items = get_influence_matrix_for_report(report_id)

# if not matrix.empty:
print(f"\nМатрица для отчета #{report_id} ({len(problem_items)} проблем):")
print(matrix.head())

# Использование в оптимизации
A = matrix.to_numpy().T  # Транспонируем для модели (проблемы x статьи)
print(f"Размерность матрицы для оптимизации: {A.shape}")

c = np.array([p[1] for p in problems_list[1][1]])
B = 10**6
a = A
L = np.array([B / len(a[0]) / 5] * len(a[0]))

res, d_mae, d_mse, d_rmse = dp_budget_allocation(c, B, L, A, K=100)
v = c / sum(c)
print(f'распределение методом дин программирования = {res}')
print(f'B sum = {np.sum(res)}')
print(f"Средняя абсолютная ошибка (mae): {d_mae / B:.2f}")
print(f"Среднеквадратичная ошибка (mse): {d_mse:.4f}")
print(f"Среднеквадратичная ошибка (rmse): {d_rmse:.2f}")

result, mae, mse, rmse = distribute_budget(c, B, L, a)

if result is not None:
    print("Оптимальное распределение бюджета:")
    # result *= B
    for j in range(len(result)):
        print(f"Статья расходов {j + 1}: {result[j]:.2f}")

    # Вычисление ошибки
    print(f"Среднеквадратичная ошибка (mae): {mae / B:.2f}")
    print(f"Среднеквадратичная ошибка (mse): {mse:.4f}")
    print(f"Среднеквадратичная ошибка (rmse): {rmse:.2f}")
else:
    print("Задача не имеет оптимального решения.")


Матрица для отчета #2 (10 проблем):
problem_item            Высокие тарифы ЖКХ  Засилье рекламных конструкций  \
budget_item                                                                 
благоустройство                        0.0                            1.0   
водное хозяйство                       0.0                            0.0   
высшее образование                     0.0                            0.0   
дорожное хозяйство                     0.0                            0.0   
дошкольное образование                 0.0                            0.0   

problem_item            Наркомания среди молодежи  Недостаток больниц  \
budget_item                                                             
благоустройство                               0.0                 0.0   
водное хозяйство                              0.0                 0.0   
высшее образование                            0.0                 0.0   
дорожное хозяйство                            0.0         